In [ ]:
import os, pathlib, zipfile, re
import urllib.request, requests
from IPython.display import HTML, clear_output
from urllib.parse import urlparse

import IPython
from google.colab import output

!curl https://rclone.org/install.sh | sudo bash
!rclone config file
!wget https://gist.githubusercontent.com/pctechkid/3f919f42bf69d8b1e91bca75d0ebca4e/raw/9f0b160fa7838d40b17396b08dd1e4b8c2412cea/aria2.conf -P /root/.config/rclone/
!chmod 777 /root/.config/rclone/aria2.conf
!wget https://gist.githubusercontent.com/pctechkid/9e4901d8dfca0895623e1eb6369a2fe5/raw/498e25555152697da9d82383fd1e7bbc35e04016/rclone.conf -P /root/.config/rclone/
!chmod 777 /root/.config/rclone/rclone.conf
!wget https://gist.githubusercontent.com/pctechkid/de60f085d6202ba087c0fa77598ce974/raw/ce665f15f5a61a84f850c6232da0e2bf1f651c36/transfer.sh -P /root/.config/rclone/
!chmod 777 /root/.config/rclone/transfer.sh

#####################################
USE_FREE_TOKEN = True 
REGION = "US"
# Aria2 Service
Aria2_rpc = True
Ariang_WEBUI = True
#@markdown <img src='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAR0AAABWCAYAAADygkJgAAAABGdBTUEAALGPC/xhBQAAAAZiS0dEAP8A/wD/oL2nkwAAAAlwSFlzAAALEwAACxMBAJqcGAAAAAd0SU1FB+YECwsKFMIooOgAACAASURBVHja7X15XFRV//97AHFBwV1TUCNxTROX1G9hpT6ulGZPZU9PbpWWu+mTe6Zm+ZSlqeRu+pRrZipSiiIuCLiggKAIbuzDvs0MzPr5/THnXu/M3NkQkOk3n9frvGBm7j3nc889530+53M+C+AkJznJSU5ykpOc5CQnOclJTnKSk5zkJCc5yUlOcpKTnOQkJznJSU5ykpOc5CQnOam6SfKU769qolraT0/C39NoW+JA71HinMY1Ng+qhNyecBK4CD5L7BgUVMUdSyJ1PmkbEqP/JSLfVYZHspFHiZm2JXb2j61t2vteq2OSOAKvtR1oCICuNgOQWyUnogsrroLiUsOgoxMMUp1IqWxbEqPB7CJSJHaAjk7wV2v0WYxH4QQS62OJHf2jE7SrFbRJFp7ZuD2XGhiHjsRrbSVh32kF/SmpbcAjqcS1rgDqAHAHUJeVOmYmRR2RNlRPiOZkBDZaAGoAGla3hn0WTnB7pAohoLqxUkfw19UIeCQW+ORevEbAl1rwWSsCQNyEcmN9zPWzu6A/XWzoI62gTyrYX7F+kQjqrCNoy9J7rcrV2VF4rc0SjnAeKFkR9mGtknjcKgE47gDqA2gE4H4Nri7cpFUAKAVQACALwAMAtwDcBCADUM6KUtDpWjOIb06q4Ca7B4CeAAYA6AygI4AW7NmtgXcpgDTWR3EAYgFksElVYWFycYBTD0BDAKsATK7kYJzE+kbGipz1i8YI6FzY8zYA0ApAQiXfUTGAxg7C69+NSgCMYn1XCqCMzQOVAHgcStKRGL3s3FooyuYAWAPgOoAi1ukKAerrjHQGQiDlSl022bsDeA/AGADNqpDH6wCCAYQxHuVsYFQw8NGxfq0L4HsAU6poJfwQQAyAQtYvXHskANmWAB7WglXbUXitzeAzAkA+G2MKo/H/1MnVDtDhVl8vAPNqYWc3BDASQCCTgIpExHiISDbuAqmiF4BvAawE4M8AtiqpDYDXAIxlPOQa6Yo4fhoCGML4qYqFpSuAKJGtJwTvtQWAT2vBIugovNZWqgfgtMh2XldbGHS1cSBwe+gGbMtQm/fMDQEMZHwqjPQrZKQzqSvYQq0D8CWA52qAx/oA/g/AG+xzHuvTOmzQfAlgYhW214xtfTLYIBSK3G5McrhTS96fI/FaW2k4gFOCbbymNm2xbN0iSQTbD7u3VXXr1q3p52rMQGQdAD8AzQF4ssnOKR4bsAG8EsAZAMOeQv83BTAfwC+sfW8Are2QQO2hSQA6AGjCgNkdj5Xj9WrZpHEkXmsjebGx1ZCNddfapA5xsfPaOmI/aLVaEJHZUlFRASKCTqeDr68v/P39LV4vLGq1GgqFAvn5+bh//z6io6Nx+PBhrFixAsOGDUOdOnUs8dwHwFoAvkwk94JeCezJti7HAXxibZK7uLhgwIABWLp0KQ4cOIDY2FhkZWVBJpNh4cKFAIBt27ZBLpcjKysLly9fxs6dOzF58mS0a9fOlr71Zjqcz5ikZfJQW7dutam/dDodXnrpJbE2PACMZgDsJQBg7pTMgDw9PQ3eXWZmJoqLi0XbLCoqckheBw0axH8fFRWFX3/9FU2bNoVarTa5r7i4mL9PoVAY/LZ27VqUlZXxn8vLy0FEyMjIwL59+wyujYuLQ1ZWlihvv/zyCwCgVatWUCqV/PdJSUlm+1DIl0gf1mNjyeFMCTixvzGTGoyPrcke6tixI/Xt25eqirKzsykwMJDE+BKUM2wQvwi9kvjf0Gv4Ld7XvXt3CgoKovz8fLPtL1u2jADQwYMHzV4TFRVFU6dOpfr161vjk6A/7Tpi/L23tzcpFAqb+uT27dvk7u4uVrcWwFymV/Jjkl5rAP2Mr23cuLHZ+jMzMw36ZM+ePQ7Ja8+ePYmI6Nq1a/Trr79Sz549KTIyksrKyqigoICSkpKIiOjChQsUHR3N35eenk5arZa0Wi0REUml0ioZyydPniQA1KFDB4PvNRqNxfvMjKO5bJvlx+ZuHTiQKYEQdDrXNtAhIioqKqJRo0aZG7xcWQ1gHIAF0J9omb22W7duFBISYlPbtoAOR/n5+bRw4UKqV6+eNeDJEft+xYoVZp/fmBYvXmyu7nMA3gLQl21hfAD0t2UiZ2ZmirZfXFzskLy+9tprtHDhQmrevDkBIIlEQhcvXiQiory8PP6emJgY2rRpk8F4T0lJoYyMDLvHalpamt2gw9GNGzfsAZ350B+hd3Z00OlSG0GHo+joaGrVqpW5l3AXepsXtbnJXq9ePVq/fr3VlaWyoCMceK+//rotUg9fBg4cSPXq1aMHDx7Y1IZMJiNvb29z9X0O4B8Anofe9ugle6QHawPfUXgNDAwkuVxOrVu35r/jJvb9+/dpyZIltH//fpP7hJSammrw+dSpU2bbPX36NB04cKDSoCMknU5Hp0+ftgY6o9mcrVWg87cyG+/fvz/CwsLQtGlTsZ87sRchahDZrVs3XL16FXPnzoWrq2u18unj44MTJ05g3759aNSokU33HDhwABKJBHPmzLHpeg8PD6xZs8bczyPZKZEXU6jXsaXOtLQ0pKenAwDi4uJQVlbm0LwWFRXh0KFDWLp0KU6dOoVRo0bx7z4lJQVr1qzBe++9BwBYu3atwb2FhYV6ja2Xl+Gx0fDhovzodDoEBARg/PjxlR43MTEx+PDDD7Fu3TpIJBKkp6dj48aNloQFhyWHkXQ4OnHihF1SxJAhQ6i0tLRSbVVG0hGuiMnJyfT8889b5ZGIaPXq1QSAjh07ZlMbWq2Wevbsaa7O5QBeB/Ay2/tblB6ioqJIoVBQeXm5VUnHkXjt3r07BQcHU1FREf38889ERPTo0SNSq9X8doyIqKKiwuD5ZDIZZWdnV+m4tSbppKSk0IIFC2jJkiVERJSQkECZmZnm+mxBbZV0/pagQ0T09ttv2wQ4b7/9NimVykq3U1nQEU4IqVRKzz77rFXQKS8vJ19fX/Lx8aGysjK7BrJIKYDewO51puuyOJE5pakt2ytH4hUAjR07lkpKSignJ4diY2OpqKiI7ty5Q+Xl5dS3b19at24daTQag+fT6XR0/Phxk7pjYmIMPqtUqkqBTmpqKmVlZYlep1KpKDY21ppOxwk6NQ06iYmJVgFn1KhR/IpWWZo1a1alQMeY7t69yys0LfUxJ8XNnTvX5rpffPFFc/WGQe/XNbEqdTqOxOu//vUvmjdvnsHvycnJRERUVlbG/2+s01EqlZSdnW3xZFOn09GWLVsqLenYqlt0go4NoOPn50dKpZI0Go1J0el0VQY8/fv3NzuJ+/XrR3K5nL/2zz//tLv+48ePU/fu3QkAbdiw4Yn5PXPmDLm4uJC1Ph45ciS5urrS9evXbar3p59+sgS+awDMrsxELigooPT0dLOgU9t59fT0JKlUSlevXjW4tqKiwuC4+vTp03T8+HH+Ps4cQKfTGZxgKZVKA5DS6XQ8cNy6dcugDTEl+6+//mqzItkJOnaCjjUJRCKRUIMGDcjX15feeecd+vLLL01OCWyhL7/8UrR+FxcXs8eP1qiwsJB27dpFQ4cOJVdXV4N627RpQ+PGjaMTJ07YdQImtl2z1MfJycnk7u5O/v7+VrcSnDg+ZMgQc/0dDmBFZaWHhIQEi6BTm3nt3bs3tWnThmJiYiguLo7eeOMN/tpNmzbR8OHDydvbmy5fvkwlJSX8fa6uruTm5mZSXFxcSCKRiP4mdq3xdxKJxAk6Twt0xEr9+vUpODjY7JGr2CT/888/ReuaPHlypQBhw4YNVLduXZv47dKlC508edLuNtRqNXXp0sVqHy9ZsoQA0O7duy3Wx62o+fn51LZtW3P8brJ1IpuzM3FEXrt27UpERNu3b6dPP/2UTp06RWfOnDG4ViqVUkpKCn3zzTeVGreVKU7QqSWgA4AaNWpk14lBSkqKSR0NGjQwq6ATI4VCQQUFBfTGG29UiucZM2bYragOCQmx2scymYx8fHyobdu2Fk/ejhw5wv8fHBxsjk8T2yUPDw+D7WdldDq1ndfRo0fzerm8vDw6c+YM9enThz755BNSq9UGW32tVusEnf8fQQcAffHFFza3VVBQIAoC9ko3TZo0eSKeR48ebdcphlg/idHhw4cJAH300UdWpQeORo0aZdNEBkCRkZE2H3k7Iq99+vShKVOm0IsvvkjBwcH0xx9/UGxsLEmlUkpLS6PZs2dTaGiotcntBB0HNQ7kwnZapT///NPmSo29211cXGw2WIN+9uDEiRMmDo32UkhICD791L7wL2PGjLF6zdtvv41nnnkGO3fuxOrVq0WvGTRoEH766SeEhYUhJiYG7du3F7vMDfrwBwb08OFDJCYmWuThxo0bUKvVDsmru7s7Dh06hPLycnz88ce4dOkSunbtilatWsHHxweLFi1C/fr1ER0djdOnT8NJDijppKenm5N0dgPYAn0Ara8AfM327ufFXBVsJalUanBvYGCgXSvIli1brK1KJwFshN6TfYMYv0Ll+OXLl622WVJSQrdu3aLY2FgD1wAxO58+ffrwvzdt2pSmT59OQ4cO5VfnjRs3UuPGjcnLy4smT55M8fHxlJ6ebk5RayJB/Pe//7VpO2vse+UovAYGBlJAQACFh4fTiBEjaPv27eTv70+zZ88mmUxG9+/fp5KSEiIiksvlTknnb7a9+okdhy4AMIOVBQyEKt1eQkKCwX2//PILEek9m61RRUWFJX+gQgCboQ+2NRfAx9CHFp3GQEjUobRPnz5WT3AuX75Mubm5REQ0c+ZM/t7k5GQqLy+n0NBQGjRoEPn5+REA8vPzo2nTpvFe076+vtSxY0eaM2cOERF9//33dObMGbp//76BsvqZZ56xOvhtPeZOTk42uM9ReG3UqBG99dZbpFarKTY2lr799lvq2bMndejQgQebmzdvOrdXVUxutQTYNNBHOONiGjeF3lfqfeMLbfVVAoA7dx4HmHNzc8Po0aMBAM89Zz044C+//IKMjAyxn4Khdx7Ng94bvAD6YNha6GO95LDflhv7CcXExODQoUO8P48Y/d///R//f1xcHP9/QEAANBoNCgoKsHLlSqSnp0OhUGDUqFGIjY3ln/fGjRt47rnn4OKi3zl/9tlnBv4/b731FogIcrncah8kJCSgT58+Vq/z8/Mz+OxIvF6/fh3h4eHw9vbGokWL4O3tjYEDByIhIQESiQTJycno1asXnPT3k3RioQ9BmWttFejdu7fNbc2bN4+/76WXXrKLz9dee02sfR2AHwH8B8B4AK9CH+r0OehDL/hBH1t5GNsimtTRq1cvevTokUl7crmcli9fzp+YbNmyhTw8PPj7OGvlt99+m/cPIiI6cOAAxcbG0oEDBwgALViwgG7dukUajYbu3LlD69evp/bt24uaIHh6elrsa0se05ZWW0fhNTAwkN/eHT16lBYvXkxBQUGkVCrp/v37VFhYSJGRkU5J528q6bxg64UTJkx4LHIEB+P111+3qATmqGfPnjYzk5eXhwsXLojqhKGPsp8NffD3HOij71cwQHJl0pqa6XeGQh90iqfY2Fi8+eabuHTpEjw8PJCUlIQ5c+agXbt2mDVrFsaMGQMXFxeMGTMGubm56NixI7Kzs7Fr1y7cunULb775JgDwSlbOa9nb2xtdunTB1q1bsWPHDuh0OhPP6vr168PHxwfbt29H//79cfbsWbP9J5QW/fz8oFKpoNFo4OnpabX/HInXF198ERMnTsS1a9dQVlbGe6ZfvHgR27Ztw6RJk+Dv74969R5HSZ0xYwa2bduGCRMmQCqVQqFQoEuXLnB3d4dSqUTXrl35MRgXF4fy8nL07t1bNGxvUVERfvrpJ+Tl5TnFm+qQdOLj4+m5556r9ArwyiuvGPhKmfMiFnMe3LRpk0XlIhf9LTMzk4KCgszx8AOA6dCn+OgOfTQ7DzxOhFcX+nCobRiQfiQ4mePL5s2bSSqVUnBwMIWGhlKjRo3oo48+oqtXr1J2drZBzJUPP/xQtI/DwsJo6NChNHjwYPLx8aGmTZsa2CKdOnWK/P39CQCdPHmSzp49K6po5a4xJz3IZDKb/NMsjYfaymtgYCCdOnWKYmJiaOjQoeTn50ehoaG0ePFiCg4Opnv37pFSqaSsrCy6fPkyf19ZWRktX76cli1bxteZmZlJFy9eNKuzO3r0KI0ZM8bkuf766y+TsepUJFcB6Oh0Orp+/ToVFhZSgwYN7AYbNzc3+vTTT0mhUFBERAQlJiZaNaV/4YUXDOo4fPiwzS9x6dKlYnzkMcXx+9BHr/NhAMPFoBXmBvMC0B76jA8XjOsS2n5IpVKaMGECDRs2jN566y2TZ1u0aBEfImL+/PnUrVs38vf3p6CgIDp48CCtXLmSzp07R+Hh4TR37lzejWLr1q0mwGxsoVtWVkaHDx+mFi1aVOn2ylF4NT7NnDJlCqlUKtq0aROp1WpSKBS0efNmk/teeuklysrKopKSEkpKSqKHDx/yTsbWrM2jo6Pp+++/t9h/toJOcHCwgfe7E3TMkI+Pj83WxwEBAbRq1SpRHYgl+s9//mNSn3AQc0ZkOTk5osf6vXr1EuPpD+iDgL0OfRS7FtAHvnaFYaZQV+gDibdk+h6TY/Rx48ZRcHAw3blzh9LS0kzCIRQWFtK9e/do//79vDtA165dacGCBXTy5Ek6dOgQnThxgsLCwohIH73uypUrVFJSwsex8fT05H2NOOIssY1N/Xfu3FmloOMovHKRA69cuUIzZ84kT09Pio6O5uPqyGQy0ftWrVpFFRUVpNVq6d1336XMzEy6d++eaCjWy5cv8/Vw94SGhlLdunWpQ4cOFBQURGVlZZUCnbCwMJLJZE7QsUWR3K5dOzp37hydP3+ejh8/TufOnaN169bRlStX6NatWzwYKJVKOn/+PK1du9bAJF2tVvNBs43p22+/FX0JnGdwSkqKwfVicW7Hjh0rVsdetrUaAn3YTHMvUsKkHS6QfbhxXdOnTzcAxhMnTpBaraaQkBD67rvv6ODBg3Ts2DHKz8/nt1fcEa6QDh48SNevX6fk5GTeg/rRo0d8vcOGDRPto2vXrhlMbqlUKhpSorKg4yi8CiWdsLAwCggIICKi0aNHU1FREQ+URETnz5/n7+MAUWgBnZGRIfrcxlEJOBDduXMnSaVSKikp4b3LnduragQdYTydc+fO0dKlS+nevXsmK9qMGTMoJCSEPvnkEwOHTqVSSREREQbXl5aW0pQpU8xKTsK9dk5ODm3bto327t1LEydONBDrFQqFOb3Tj9CnvA1gW6dGTBEvBjpu7Pdn2ZbMoK7OnTvT77//zp/0jBw5kvr27UvDhg2jCxcuUEVFBZ06dYouXbpEmzdvNtvHCoXCJBSIsUGkpZAd9+/fpz/++IO0Wi1du3bNJKxGZUHHUXht0aKFQSgK4Vh69dVX6cGDB3Ty5Em6f/8+TZgwweT5YmNj6eLFi6TT6UTrEW4NVSqVSbD4X3/9lVc/VAPozHeCjhnQUSqVJiuERqOhMWPG2BTYW6lU0oIFC6wakAkVfklJSRQREcGHpujXrx9NmzaNNm3aRO+8844lJfJE6LOHejMFsiXQaQignRjojBs3zupzlZeXU0hICE2aNInnX6FQULdu3fhg4TKZzMTzPioqyqCtTp062exwOnXq1CoDHUfgtW/fvnzWh8uXL1O/fv2IiOjixYtUWlpqoBcURhfggOLcuXNUWlpKSUlJNHjwYLPOrB988AGpVCqTgw/h9q2yoPPHH384Qcde0DG3KopRVFQUjR49mj+huH//Pk2aNMls4Cth4RwuOYkpMzOTunfvTo0bNyaJREJeXl60aNEiysvLM5efajuACQAGMNBpYKTPMQYdD6ZsXiemr8rKyrLq7f7bb7/x2S2EJ21qtZoePnxIc+bModTUVIqNjSWNRkMymUzUQ33NmjVW3wu3dRGeLD2ppFPbeeW2VzKZjN5//306duwY7dmzhzp27EhLliwhnU5Hffr0oYiICIqPj+fv+/HHH2ngwIFkL4kpkDldjxjohIeHi15/+/Ztio6OJp1OR2lpaQ6XgqZaHT6Ftge5ublITk4GoM+QmJKSAgC4d++eyX2HDx8WrW/AgAFYuHAhbyns6+uLn3/+GdeuXUOHDh0s8hIWFgYAKC8vR1xcHM6ePYuEhAQkJCTg1KlTSExMxIwZM9C8eXMMGDBArIoWeJyeVSIolsBa1BZq4sSJePToES5fvozs7GxoNI/9F7ksAyEhIbhy5Qpyc3MN7vXy8kJGRgZat26NH374AS4uLnjhhRfg6uoKDw8P0YyZX331FR4+fGixf+rWrYtWrVpZyspgNzkKrxqNBsnJyYiJiUHHjh3RvHlztG/fHjKZDFOmTEGXLl3Qo0cP/vrMzEyMHDkSOTk5kEqlonUWFxfj1q1bBt917txZ9Fqt1jTNuFKpRP/+/XnbLo5SUlJQVFQEHx8fTJ48GREREWJVZhoBUK2iajMOLC4uxsmTJzF58mQAQMuWLdGyZUsoFArodDrecEsqleLs2bNo2LAh2rZti9atW2PixIlm6w0ICDD5rnfv3jh//jz69++PnJwc0ft27tyJwsJC/Otf/8ILL7zAu1O0bdsWbdu2NbjW398f4eHhxlW0xON82rbkhpYIAEonvL5JkyYoKCjAP//5T8THx+Po0aOYMWMGAPDpc0aPHo1hw4YhOzsb+/btM6iYA9jc3FzoF27LVF5ejhkzZlj00OfanTp1Knbs2IEbN25Ynfy2kCPw6unpic6dO6NTp07o1q0btm7dimXLlmHw4MGYPn26yfV3797FiBEj4O/vj1dffRXvvvuuwe8KhQKRkZEYNGgQ4uLiIJfL0aRJE5SUlODgwYOoX78+tFotMjMzkZOTwxskCikyMhI5OTkYP348evXqhRMnTuCNN96An58f/Pz8sHr1auzduxfNmjUz91i62gg4NbK9unLlCm3bts2u7ZUt9gliFBoaajGmzQcffGAQlMkcbd++XayOSOjj8w7D41St7kbSjsTo9OpFAIeMDQRXrVrFbxHT09MpMzOTtm/fbuBcyMWgmT17tl1bWLFsm1yxFDxeaFIQFRVFEomEevToYXPM6spst582rz169KDffvuNNmzYQLt376bNmzfTe++9x+tKuCPw8PBwE2VvdRZ3d3fe6Vd4YiuXy0mlUlF8fDwNHjyYNBoNb3ZgVM7ib5RWuNp0Ota8r4ODg21KZfL++++LvsigoCCTE4qcnBxKTk4mqVRKMpmMEhMT6dy5c3T27FlzA+IrAGOY/U1LZo/jarTdcmX6npYAVopZJH/55ZdUWFhoU3+NGzeO7+NHjx7xx82VmcitW7e2Ob/X5MmTCQB/wmIv6DgCryNHjqSCggL666+/eK9+LhY3d4hx584dXgdYU6ADgPbt20fbt2+nnJwcunnzJt25c4fmzZtHcXFxvC1XfHw8ffTRRw6Xy/ypBPGSy+UoKyuDXC6HXC5HeHg4vv/+e8jlcrMieHZ2NnQ6ncV64+PjsWzZMtHfjhw5ggMHDuC9997DiBEj0K5dO3z++ecoKyuDQqFA3bp10bx5c7z22mtQKBRwd3cXq2YwOwpvyACnjpEy2YVtwdwZ6HQ2ftGtWrVCWloaCgoKbOqr27dv8/+3b98ezzzzTKX7XSqVmg2gZUxr166Fl5cXFi9eDIVCIXpNWlqa2fsdgVdXV1dIJBI8++yzAIBNmzahXbt2AMB/16VLF17vU1PUoUMHvPvuu1Cr1cjKykJubi7u3LmDqVOnIigoiN+iu7m5YeTIkdbmt0Nm+qzRcKWurq7Upk0bGjx4MK1YsYK3bXj06BFvbm6OpFIp3blzhwICAkzqbdmyJS1atIhCQkLo4MGDdOPGDUpNTaWtW7fSokWLaPLkyfTxxx/zdQ0bNkyMPyX0+dBHMavk1gyAhL5XXtB7nK+B3nXCoI758+eTVqsljUZj4MEsRnl5eXyGgKrYsnCi+927d22qa+PGjQSAzyhZk9urmuCVO72qqKjgj86FvEVERFB6ejr5+vrSxYsXa0zKadKkCW+hHRkZSR9//DENHDiQrl+/zm8hhWlyrJxedYE+VExdEam8suXvBTpiL0B4dCg0EpTL5Qa2PFOmTKFu3brRv//9b9HIfXv27CEifezdjIwMCg8Pp/3799Nff/1FkZGRNHHiRN5AbcGCBeZ4SoXekfNV1h/PsL7xhD7ntg8zJBS9f8+ePRYNyYTEhYCoyokMlmRQ6BcnZpXN9XXPnj3J3d3dauCz6gCd6uaVA53w8HD64YcfDPQnnCuESqWixYsXW5rcF6B3dQkHcM6ohBqVcyIlDPp4UgbRMSdOnEgFBQX06aefUsuWLQ1cZRITE/mgZdu2bTPH13+gd9npzk5eGzDgqVOJ4saK8eltpQHFlms4u5PW0Me9MSBbTiU48vPzEz0mt0QeHh6Ij4+Hr68vbty4gd69e/MnIi1atIBEIuFF6IyMDCQmJmLq1Kkm9axbtw7l5eVwc3NDz5498f3332Py5MkgInzwwQcAgIqKCtSrVw8qlQq9evUyCAQmoFRmLBgPfRRBGTstqAe9q8Qsc8+SmZmJNm3aoKyszGpAsjfffBPHjh2zq49LSkrQuHFjq9edPHmSD2pmiS5duoRBgwYhICAAFy5c4PvaZJAIvncUXgMDAxEcHAyNRgM3N/GD3KioKAwcONDkGQX0K/ThTcoAlBsBCEROkIQSQx02ZhYZb69mzpyJnJwcfPvtt9BoNHB1deX7q6KiAgkJCejZsydkMpm5E6ylAFIAPGQSN8ebrhI4wcWS0gqKzsIzOrakw5V///vfZrcgnLiZlJREeXl5Zg2m+vXrR0VFRRQSEmLVKzgtLY3CwsIs8aQG8Bf0wbr+wV5yrLnrPT09aefOnTafsEilUnJ3d68W6QEsdKit1r+c5GgpRW51STrVyWtgYCBpNBpKGSYHkwAAEzZJREFUSEgwcdaUy+W0YcMGWrlyJW/EaYa/XQC+A7CEKW+nQ593/RNWphmVT9jvMwDMA/CF2LbylVdeof79+9PcuXMpKSmJsrOzKSUlhUJCQnijSyvbq9XQu+0Mhz6m0/Ns/tpTOkMfwbMjUxm0YdJ8IyY1uVXXlqsmQEcmQE7RYi2L44ULF4iIaPXq1fyJj1ixFn5ASJcuXaLXX3/d2qSIZuCjNndNnTp1aOHChZSXl0dqtZpWr17Nt2F8TM7R8uXLq3Iil4jxtXbtWpvqzMrKokaNGlGjRo0oOjq6ukGnxng1Dm2Rnp5uoDc8fvw4jR8/nt8Oo4YjB+bk5PCnujt27KBdu3aRVquls2fP2uJ7VVWlnJVdAPoyDPAG0IQdplQL8FQJ6HAeuWZAZxeAIAD/BfA9m8gm13Erzo0bNwwUaURESUlJFBISQosXL+Ynulgd06dPN7hPpVLRiRMnzPL98OFDatasmaWXorX24r755ht+EF24cIG0Wq1Fv7KioiKTXFvVMZEbNmxo4oRoyYQfFrzBHZHXwMBAiouLoy+++EI0zInQvOKvv/6qUdC5efMmffDBB7zv2rZt28wePNQQXxz47AbQB3pn5mZMV1TlwFMloMPlDjcDOhuhz1E9i4mgn4m1wwW/ysjI4ENb7Nixg6Kjo/mQFffu3aMvvviC5syZI9p5devWpStXrljl99GjR7xI/+DBA+rfv7/dL8rFxYW+/vprA5HdXEgOIXFpeKt4Il8S45EzhrNGKpWKunXrRgD47Wk1gk6N8BoYGGgQOsMSFRUV1Sjo7N69mzZs2MArto1JrVbz39ek/RD0caVmQh/IzpedjBnHlXKI7dVathceB2Ak+5ttfN369ev5ejgjroiICDp+/LhJLvO9e/ea7TjjwRsREWESqS43N5dCQ0P5XFVqtZq++uorAz2LpdK/f38eaO2hlJQUqlevXnWAzl6YSY1z8eJFm+rmdFxdu3Y1mQiOyCu3vbp3755NUlRNb6/MUUlJCalUKnr48KGl06vqlHguAtjJtlvtmJmIe1XaBdUE6KwC8AE7hu4Lfcya88bXTZkyhRITE61aLxMR3bp1y2Ln7d27l27evElardZESclZoYopfS9evEi///47zZo1yyBbA1hY1Y8//phmz55tU6xeMTJjH1QVE/ln5pJh8nvPnj1NQNsccaE/hGE8qwF0aoTXQYMG8YuOJdq7dy+Vl5fXumwQnL1XDYMOAYhgC8NIAD3YqbZwm+UQoLMCwNvMV6kLczPYBZH0M2FhYRQTE8NLJkuWLOFPHoSitFqtFpUYuOLt7U2pqalUVlZmEi6UG9Rbt27lzc6FL/r999+nlStXmoCOq6urxVzd1mjr1q1W4wE9wUTexE7assTq37hxo031p6enk4eHBzVv3pyKi4urC3RqhFducpeWlprV68lkMkpJSRGd3Ddv3iS1Ws3HyeF0jlFRUURkGPVQLpdTVFSUgRK4oKCA9u/fTz/++CP/nb+/vwHoqFQqSktLMzFeFOqbYBTeIy0tjUaOHElnzpzhQ7pw7iIpKSn8oqrRaPiEBPn5+aRUKvmElPHx8VReXk4//vijuTF5iS0OAWybJeaLWKtBZznbUvmzB+gEYLGxiO3u7s5LJQUFBVbbEgtrKSzvv/8+nT59mj/5Wr9+vcFp0tWrV+nDDz/kP8fFxdE777xD+/fvp4yMDNq3bx8PbFOnTiVvb2+bDf+MKTY21gTEqngirwXwOcxkTW3cuLGJk6E5+uabb3jr6moCnRrjdfHixfwk5JT9xluZf/zjHyb3jRkzxmybwowPnP6wuLiYEhMTzRo3ciDg6+tLHTp04PU5ycnJFiV7zhIbAD+OxSg1NZWys7N5ZbRWqzXgxVx/7t2711wOuAIA/wMQCMOY4S5PFXSOHDlislc2AzrLGOj0YnvEZ5nkY7Kvj46OJo1GY4DgRETR0dEGOp+7d+8a5M+2pHspLS3lpZuHDx/SJ598Qr///ruo7cz8+fMNYjh7eXnxdc2bN4+Ki4utxso1pujoaJPTqmoAnS+Zjcgkc6eDkyZNsqkNpVJJnTp1Ind3dz7MrCPyyqV5LikpocTEREpPT6fZs2cbREW4dOkSdenSxSCX+cKFC+nevXuUmppqVgLhJjY3Th88eCBqa8RJRzqdjk+w2LhxY4qJiaHY2Fi7HFjfe+89vg2dTid6QqpSqejq1as8f5aooKCAHjx4QKGhobRu3Tqxd7UP+qST/ZgNDxfYziEkHQ50XmA2AN5Mt2MCOl9//TXdvn2bZs6caRKtnwMGpVJJiYmJllwZTIwGhZLT/fv3reoNCgoKqGvXrnzI0yFDhtCCBQvslnCOHz9ODRs2tMpjFUzkJUxvNoadEGrEXEWs6Tc4OnXqlEHoVUfklVMkC/31jhw5QkREZ8+epTlz5vBtXLhwwWBrnpWVRc2aNaPly5fzz5KamkoZGRn8QnvgwAF+AaqoqBDdFnKS0P/+9z968OAB9e3bl9q3b2/XGOL4ioyMpAcPHlBSUhLpdDqTEDDXrl0z6LPbt29TfHw8r3/kdETcdnH48OE0Y8YM+u6778yNywIAkwG8DMO44TUDOlxiMqFYaifovMl0OW2YP1N3ANeMr+VsL3Jzc2nbtm109+5d0cRsXHoXM3m2TAZx586dKTQ0lHbu3El3797lT8fMrZ5Hjx4ltVpNAwYMIAD04osvUqNGjUwCzBt/ForSX3zxBe/QWQOg8zmTHgdDn3n0f0+qqH3zzTd5sd4ReQ0MDKTjx4/T7du3KSUlhVxcXKh9+/a0ePFiat68OcXGxtK2bdsoKyuLYmJiDDKLKBQKAkABAQH84pebm0u3b9+m8ePHExHRwoUL+fAeZ86cIaVSSXfu3KHbt2/TkSNHDMKbbN26lR49ekTdu3c38BMUHmDI5XIT/aMQdPbt20fPP/+8gT2PTqcjnU7H6z0fPXrEO9EWFRVRTEwMFRcX09dff00//PADLV26lD9AmTNnDh+DyswCXsCsqwcztYhnjYKOkHJycgyyVNoJOs+wsBB+zGDQ4No2bdqQUqk0CHItVMRxoiyH3D169LBZK9+oUSM6ePAglZaW0rlz5wwkH4VCQRs2bKAtW7bQtGnTaPPmzdS5c2c+x/b48eMpMTHRbHxnISUkJFjVN1UD6MwHMJbZV/RlzoB5Ym0JnR+t2TLVr1/fIFaxI/H66quvUlBQEB04cIBSUlJo7Nix9Morr/C/z549W5RXTsHMGaAab8Xv3r1LkZGRVFpayutKVqxYQbm5uXT+/Hki0meImDBhgsERPxFR+/btydXVleLj43lgyM/P5wGEc0TllNNC0AkNDaWIiAjecFYmk9HKlStJpVJRQkICb6HNqSEuX75M169fp8jISJOt3P79+6lOnTrk5eXFK5rNgM4s5gbUkR2d1xzocA9mvPIUFxcbHD3bADqtmaVjewAfi7V3+vRpk0lMRHTo0CEaN24c9ejRg3/5QUFB5ObmZpO0I/Rs5twluL+//fYbZWZmkkKhoK5du9KePXuofv36/Aqwb98+XsIy51+VnZ1NM2fOtNnWpxomciBT1ndhk3mFOetfSwG4srOz+fe8atUqh+a1uLiYzp07R/Xq1aPmzZtTt27dqFu3bvTPf/6TX7yM/fm4Lcr69eupQYMGJiecQuUt1/bLL79M6enpBokjhSdqR48eJaVSSW3atDE5MtdoNLR27VqSyWQGknNZWRmtWbOG5+uPP/6g1NRUvj8qKioM+iYxMZHy8vJowIABRKRPgJCdnW2iHyXSR60EwFtE79692xzozMHjaJpeDCtqBnSKiooMFFNpaWmk0Wjo+vXrBt9bAJ2x7My/JWurDRPbTPQ6Y8eO5Q3pfvvtN77uefPm8RP+0KFD/Peff/65WJuFtiiZPT09ydfXl18J5HI5+fj4UKdOnejzzz/nt1dceE25XG4COlFRUTRlyhRzWSbEQiVUx0T+DPqUJM8zQPdje/ErMBPeNSwszKq9UUVFBfn6+josr19//TUlJSVRbm4u/11mZiZt3brVxHKd+51TI8TGxlLnzp150Lh06RK/1fr5559p8+bNNHz4cNqzZw/t2rWLl1gsUd++fcnNzY0/RT148KDJSa1UKqX8/HyaNWsWrVy50uCQxRodO3aMwsLCaNGiRTRp0iRatmwZv2g/ePCANm7cSIcOHeKzjnIUGRlpDnRmVzfodBZ76cK9qdiek0ifIVGlUlHr1q3Ngc4YNshaMoVUCwDdWMwRE/cCoQ1EUFAQvfDCC7RixQr+O+EJhBn7l1MAomwBHwD08ssv07fffktxcXE0bdo0mjZtGhERjR8/ngDQzJkzqaysjDIzMykiIoJ27NhBU6ZM4dMC21A0APYD2CL2e3Z2Ni9OV3Iiz2XGXF2YNMnpzSaYk/qM9QrC0zbhBOdWRUfjtU2bNtSuXTvy9vamIUOGGBzJz5o1iw4ePMg7mq5du9Zk21VaWkoTJkygESNGUFhYGI0cOZJ2797Nb+0/++wzSklJMThVDQkJsegB37dvX3J3dzeQgi5dukSHDx+2qkg2BzpKpZL3/+Ns2BYtWkQKhcLAJGTo0KE0f/582rNnD/Xr14+aNWvGXz9jxgyxdxXDXCKGAnjO1u2VvfF0WgBIroZAYUsB3ALwAEAOM7Wuw/w6AllQrKqmv1jH1YE+lxUY2D0NugsghHnbq1lYgqqkEmYLdZfFVylkXv1cjKTp7BSiNtDT4LWU/fWEk+yhG9Dng7sLfYypfEHcHrNkqyGPMIhPURUzrmZbKLUgyJCO/V8OfRaGm0/SQL9+/bBr1y7jIEzcM90AsIP5etU05TAflv0AHrFJlsq+r2pSsT7mgjCpACgAFAM4AH2upNpCT4vXUieO2EwaAFfZHFXBjgBhthjycPmbXJmZcwj0wYGqir5l5u45DNDkAhGaa1cBYERlG8jKysKJEyeMpRwO3O6wtqMYuLUG0LwGwOYg9CEuU9kkygKQywZ+KID3qnAirWdt5rI+Lsfj+D8cAEuZpNfuKQ7kp8FrqRHYNXDiiU0UC70rBPeuSgBU4HGoF7Nk6/EWJ3ko2RagCPogPk8q4YDVVQR9uMcK1o6WgZ2KgVAEAwT/KuownUByy2MrqIpN/ttsMNeH3h3DH0DXKmr3JvQWtlLBcxey9ktY/7qyiZbDJtaTTgIStCEXrErc4Chnv0UAeOkpD+SnwavK6HN+DSw6fwcpJ5r1VZHgXelskXZsdc7i9DoNGNi0ht5qeHslwIcDm/8y7bcUQAbb3hQyqUYjAEVhm22ht1btVcnO+hOPA2/pAJxg7XPtSqD3H/HA41Qz7ZhCu4+gv8RASCbyOZOVdAZuMlZKjIpMsBd2Y223ZM+7uZLgU8J0YflMispgz1rMnlUr8l5bQn9q+AH0p4mVkeAchdczbBGQCCRrsLqfZd8Pt7GuMgCJAt2go1MI9KeHYmCzi6kCctmczWRju5QtmlYlHXtAxwV6pasHezHNWWnKvhOmuLC0kmkFEkwxYzif/S9jv2kF2z93wUBrxtpszAChrkBakwj49IDe5kcsmplQP/U/NlEKGT86Vl9dJul4sNKQ/W0gaNPFzPaUjNrgdFYVrA2ZoMgZ2FQI9sRurO3G7HlbsGf3EHleS32sZPUXsv4VrkpqwYrkyt5rA+hPH5qyPm7CAKQebIsMR5UYYzXNK8cjNwZVrO+N2+Byl3GphVwF71pipk7uvQvbdrScU2L5z7nnEepZSwTvqpABjlhQ+ifaXpFRo5zEImfSCpd4zgWWldNcPWpWDzf5ytiKphYAjnBwcPeq2PXGye5cBB3EDZok6I9aBxrxpANwnA3qAva3hPGjxeOEeVweKw6A6rNB7Q7LqThI0I6W9ZlKADzlgv+FClOd4D5h8HcF45Nr21oKEJ1IH5caSVM6oz4Ga0fYZiGb3O4wzJdkD+hYm3Q1zSsZjWWViBLUVQBuXPoVV8GCZq5OsWd3VNCBAECNJR0l6/MyVuRGuhyrmSHs6RSJ0UrATUh3kYloDXS0ggdQCl6+zohxicjqU89CmxIYZtmsx8DCXbBSqQQDvETQcUIJSyIy+OoYrXpiKxqJDG5ugGvYJFELPmsF1xkr+Kuij1UCwFMK2oSZPuYkS2G7bhYmnDXgkdi4mNUkr8bpVDRGWwIXgRTrJvhsTXoiBwccS89iLDAI35Xw1JnsARJ7gUf4YlwtbDVsUeRqBf9bGrhibRpPQInRgHE3mrBC6alCsLURO/KTCOoXPqM9mQ6FwCPUJRHM5w2qyj42zlOks7GPjdut7glU07ySyLsRq18CB07NW02gZC7/FdkLIpUBHohIGZVBVJ0NegGISDNi/BgPSDdBEe5NtQKEVpsZ5BKRlUtiZ9+Rlf0y1eI+rkndRE3zSha2RxIL48oJOuJjmSoLIHhCAHrSB6mqNoXAAxHpxHiFswWpJVX80hy9j2tiYDsCr04QekoDujaSxMrKZavE4SQnOclJTnKSk5zkJCc5yUlOcpKTnOQkJznJSU5ykpOc5CQnOclJTnKSk5zkJCc5yUlOcpLD0f8D9BXxtQkeSC4AAAAASUVORK5CYII=' height="70"/>
TOKEN = ""  # @param {type:"string"}
#@markdown &emsp;<b>KEEPALIVE SCRIPT:<b><br><small>&emsp;&emsp;function ClickConnect(){<br>&emsp;&emsp;&emsp;console.log("Working");<br>&emsp;&emsp;&emsp;document.querySelector("colab-toolbar-button").click()<br>&emsp;&emsp;&emsp;}setInterval(ClickConnect,60000)<br><br></small>
#@markdown &emsp;<b>AUTO-RECONNECT SCRIPT:<b><br><small>&emsp;&emsp;function ClickConnect(){<br>&emsp;&emsp;&emsp;console.log('Reconnection in progress')<br>&emsp;&emsp;&emsp;if (document.querySelector('#top-toolbar > colab-connect-button')<br>&emsp;&emsp;&emsp;&emsp;.shadowRoot.querySelector('#connect')<br>&emsp;&emsp;&emsp;&emsp;.textContent<br>&emsp;&emsp;&emsp;&emsp;.replace(/[\n\r]+|[\s]{2,}/g, '').trim() == 'Reconnect') {<br>&emsp;&emsp;&emsp;&emsp;document.querySelector('#top-toolbar > colab-connect-button').shadowRoot.querySelector('#connect').click();<br>&emsp;&emsp;&emsp;} else {<br>&emsp;&emsp;&emsp;&emsp;console.log('Nothing to click');<br>&emsp;&emsp;&emsp;}<br>&emsp;&emsp;}<br>&emsp;&emsp;setInterval(ClickConnect, 60000)</small>
PORT_FORWARD = "argotunnel"
# OUTPUT_DIR = ""
PORT = 8221
HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/pctechkid/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

from ocr import (
    runSh,
    loadingAn,
    PortForward_wrapper,
    CWD,
    displayUrl,
    findProcess,
    findPackageR
)

loadingAn()

# Ariang SETUP
runSh('apt install -y aria2')
pathlib.Path('ariang').mkdir(mode=0o777, exist_ok=True)
pathlib.Path('downloads').mkdir(mode=0o777, exist_ok=True)

# github latest releases tag define
def latestTag(link):
  import re
  from urllib.request import urlopen
  htmlF = urlopen(link+"/releases/latest").read().decode('UTF-8')
  return re.findall(r'.+\/tag\/([.0-9A-Za-z]+)".+/', htmlF)[0]

# Downloading latest version of ariang
if not os.path.exists("ariang/index.html"):
  # BASE_URL = r"https://github.com/mayswind/AriaNg"
  # LATEST_TAG = latestTag(BASE_URL)
  # urlF = f'{BASE_URL}/releases/download/{LATEST_TAG}/' \
  #             f'AriaNg-{LATEST_TAG}-AllInOne.zip'
  urllib.request.urlretrieve(
      findPackageR('mayswind/AriaNg', 'AllInOne.zip'),
      'ariang/new.zip')
  with zipfile.ZipFile('ariang/new.zip', 'r') as zip_ref:
      zip_ref.extractall('ariang')
  try:
    pathlib.Path('ariang/new.zip').unlink()
  except FileNotFoundError:
    pass

# START_ARIANG_WEBUI_AND_ARIA2_RPC
try:
  if not OUTPUT_DIR:
    OUTPUT_DIR = f"downloads/"
  elif not os.path.exists(OUTPUT_DIR):
    clear_output()
    print("Error: Your set path not found! Create path!")
    exx()
except:
  OUTPUT_DIR = f"{CWD}/downloads/"
if Aria2_rpc:
  if not findProcess("aria2c", "--enable-rpc"):
    try:
      trackers = requests.get(
          "https://raw.githubusercontent.com/ngosang/trackerslist/master/trackers_best.txt"
      ).text.replace("\n\n", ",")
      cmdC = r"aria2c --enable-rpc --rpc-listen-port=6800 -D " \
            fr"-d {OUTPUT_DIR} " \
            r"--conf-path=/root/.config/rclone/aria2.conf " \
            fr"--bt-tracker={trackers} " \
            r" &"
      runSh(
          cmdC,
          shell=True
      )
    except:
      print("aria2 rpc not enabling. Try again later!")

# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
if Aria2_rpc:
  Server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['Aria2_rpc', 6800, 'http']], REGION.lower(), 
    [f"{HOME}/.ngrok2/aria2.yml", 5042]
)
  data = Server.start('Aria2_rpc', displayB=False)
  Host = urlparse(data['url']).hostname
  port = "80"
  clear_output()
if Ariang_WEBUI:
  if Aria2_rpc:
    filePath = 'ariang/index.html'
    with open(filePath, 'r+') as f:
      read_data = f.read()
      f.seek(0)
      f.truncate(0)
      read_data = re.sub(r'(rpcHost:"\w+.")|rpcHost:""', 
                        f'rpcHost:"{Host}"', read_data)
      read_data = re.sub(r'protocol:"\w+."', r'protocol:"ws"', read_data)
      read_data = re.sub(r'rpcPort:"\d+."', f'rpcPort:"{port}"', read_data)
      f.write(read_data)
  try:
    urllib.request.urlopen(f"http://localhost:{PORT}")
  except:
    runSh(f"python3 -m http.server {PORT} &", shell=True, cd="ariang/")
  
  Server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['Ariang', PORT, 'http']], REGION.lower(), 
    [f"{HOME}/.ngrok2/ariang.yml", 5043]
)
  data2 = Server.start('Ariang', displayB=False)
  data2['url'] = urlparse(data2['url'])._replace(scheme='http').geturl()
  ariaNgLink = data2['url']

if Aria2_rpc:
  display(HTML("""<!doctypehtml><meta content="width=device-width,initial-scale=1"name=viewport><style>.tooltip{position:relative;display:inline-block}.tooltip .tooltiptext{visibility:hidden;width:140px;background-color:#555;color:#fff;text-align:center;border-radius:6px;padding:5px;position:absolute;z-index:1;bottom:150%;left:50%;margin-left:-75px;opacity:0;transition:opacity .3s}.tooltip .tooltiptext::after{content:"";position:absolute;top:100%;left:50%;margin-left:-5px;border-width:5px;border-style:solid;border-color:#555 transparent transparent transparent}.tooltip:hover .tooltiptext{visibility:visible;opacity:1}</style><input hidden id=myInput value="""+Host+"""> <a href="http://ronnel-downloads.serv00.net/#!/settings/rpc/set?protocol=ws&host="""+Host+"""&port=80&interface=jsonrpc" target=_blank><button>Ronnel Downloads Portal</button></a><br><br><div class=tooltip><button onclick=myFunction() onmouseout=outFunc()><span class=tooltiptext id=myTooltip>Copy to clipboard</span> WebSocket</button></div><script>function myFunction(){var e=document.getElementById("myInput");e.select(),e.setSelectionRange(0,99999),navigator.clipboard.writeText(e.value),document.getElementById("myTooltip").innerHTML="Success!"}function outFunc(){document.getElementById("myTooltip").innerHTML="Click to copy"}</script>"""))